# Spring Security
This notebook contains my notes from the *Spring Security* talk by Craig Walls at UberConf 2019. I had hoped for more code demos, but there were a lot of slides. It was a good introduction to the project, however.

The talk focused on securing a REST API using Spring Security. In particular, it built up towards using OAuth2+JWT to secure the application.

### What is Spring Security?
Spring Security is an extension to Spring. It has the following features:

* Easy to get started.
* Helps you implement both authentication and authorization.
* Supports *many* security schemes.

### Using Spring Security in Spring Boot
Spring Boot auto-configuration will help you a bit with Spring Security, but not much. Most organizations have very distinct security needs, so Spring Boot can't really have a good generic default configuration that fits everyone. As a result, it defaults to simple username/password authentication. It creates a single user and a randomly generated password that it logs to the log file. It applies security on all paths, and has no authorization.

Some initial simple properties that can change this:
* `spring.security.user.name`
* `spring.security.user.password`

That's a pretty terrible way to configure username/password, however. Here's a better (but still imperfect) way to configure the username/password for your initial user:

In [ ]:
@Configuration
public class SecurityConfig extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationmanagerBuilder auth) throws Exception {
        // Requires relational
        auth.jdbcAuthentication()
            .usersByUsernameQuery("select username, password, enabled from User where userid = ?")
            .authoritiesByUsernameQuery("select username, authority from Authorities where userid = ?")
        
        // Much more flexible
        auth.userDetailsService(userDetailsService)
    }
    
    // You must provide a PasswordEncoder
    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder(); // There are many other types of PasswordEncoder

    }
}

When you are using Spring Security you can get the user information in several ways. One of the easiest is autowiring  the Principal object in your endpoint handler method.

### Security Schemes
There are multiple ways to do authentication:
* HTTP Basic
* Oauth 1.0(a)
* OAuth 2 (+JWT)
* mTLS (mutual TLS)
* Roll your own (bad idea)

Of these, OAuth is the most common. 

### OAuth
Here are some OAuth terms:
* Authorization server - The entity that issues access tokens
* Resource server - Your API that is secured with OAuth.
* Resource owner - The entity that owns the resource
* Resource wanter - The client that wants the resource
* Bearer Access Token - The token that grants access to the data

##### Grant Types
OAuth supports multiple *Grant Types*:
* *Authorization code* grant - Ask for an authorization code, then use that to ask for an access token. Also can get a refresh token back to get more access tokens when the token expires.
* *Implicit* grant - Like authorization code, but get an access token back directly. No refresh token.
* *Password* grant - Client acts on resource owner's behalf, providing credentials to the auth server.
* *Client credentials* grant - Client acting on its own behalf, not on someone else's

##### Verifying Access Tokens (without JWT)
To verify an access token, the resource server needs to talk to authorization server on every request, asking if the given token is valid.

##### Verifying Access Tokens (with JWT)
JWTs are great because the identity of the user is part of the access token. This means the resource server doesn't talk to the authorization server when verifying access tokens. It just verifies that the JWT is correctly signed and looks at the identity information on the inside.

We used JWTs at BYU and I really enjoyed using them. You have to be careful when using them, however. There can be many security gotchas because your application must do more of the validation work; it is best to use libraries for this to make sure everyone is doing the right thing. Additionally, token invalidation is very difficult using this scheme because you're not constantly checking a centralized identity system.

### Authorization Server
Spring Security provides a lot of help for the resource server side of things, but it also provides its own authorization server that you can set up and run. 

When running the authorization server, you provide a token store, as well as the list of clients to register. It seems to have a lot of similar functionality to the authorization server piece I used in WSO2.

### Takeaways

* I would like to try playing around with setting up a resource and authorization server using Spring Security.
* Once I have done that, I would like to see if it would be useful at all to create a Spring Security starter that integrates with our identity and authorization services.